### Pesquisa Serviços

In [2]:
import requests
import pandas as pd

# URL para a Pesquisa Mensal de Serviços
url = "https://servicodados.ibge.gov.br/api/v3/agregados/8163/periodos/202010-202409/variaveis/7167?localidades=N1[all]&classificacao=11046[all]|1274[56704,56705,56706,56707,56708,56709,56710,56711,56712,56713,56714,56715,56716,56717,56718,56719,56720,56721,56722]"

# Função para processar o JSON e transformar em DataFrame
def process_pesquisa_servicos_json(data):
    current_categoria = None
    rows = []

    for resultado in data[0]['resultados']:
        classificacoes = resultado['classificacoes']
        for classificacao in classificacoes:
            for codigo, descricao in classificacao['categoria'].items():
                if '.' in descricao:
                    partes = descricao.split('.', 1)
                    prefix = partes[0]
                    nome = partes[1]
                    if len(prefix) == 1:  # Categoria de maior nível
                        current_categoria = nome

                # Processar a série correspondente
                for serie in resultado['series']:
                    localidade = serie['localidade']['nome']
                    for periodo, valor in serie['serie'].items():
                        ano = periodo[:4]
                        mes = periodo[4:]
                        try:
                            valor_float = float(valor)
                        except ValueError:
                            valor_float = None  # Tratar valores inválidos

                        rows.append({
                            "Categoria": current_categoria if current_categoria else '',
                            "Ano": ano,
                            "Mês": mes,
                            "Localidade": localidade,
                            "Valor Índice": valor_float
                        })

    # Cria o DataFrame
    df = pd.DataFrame(rows)
    df['Categoria'] = df['Categoria'].fillna(method='ffill')  # Preenche categorias nulas
    return df

# Realizar a requisição
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    # Extrair e organizar os dados
    df = process_pesquisa_servicos_json(data)
    # Salvar em um arquivo CSV
    df.to_csv("pesquisa_servicos.csv", index=False, encoding='utf-8')
    print("Dados salvos em 'pesquisa_servicos.csv'")
else:
    print("Erro na requisição:", response.status_code, response.text)


Dados salvos em 'pesquisa_servicos.csv'


C:\Users\biboy\AppData\Local\Temp\ipykernel_110000\264442125.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Categoria'] = df['Categoria'].fillna(method='ffill')  # Preenche categorias nulas


### Pesquisa Industrial Mensal

In [2]:
import requests
import pandas as pd

# URL para a Pesquisa Industrial Mensal
url = "https://servicodados.ibge.gov.br/api/v3/agregados/8885/periodos/201901-202409/variaveis/12606?localidades=N1[all]&classificacao=542[all]"

def process_pesquisa_industrial_json(data):
    current_categoria = None
    rows = []

    if 'resultados' not in data[0]:
        print("Erro: 'resultados' não encontrado no JSON")
        return pd.DataFrame()

    for resultado in data[0]['resultados']:
        if 'classificacoes' not in resultado:
            continue

        classificacoes = resultado['classificacoes']
        for classificacao in classificacoes:
            if 'categoria' not in classificacao:
                continue

            for codigo, descricao in classificacao['categoria'].items():
                if '.' in descricao:
                    partes = descricao.split('.', 1)
                    prefix = partes[0]
                    nome = partes[1]
                    if len(prefix) == 1:
                        current_categoria = nome

                if 'series' not in resultado:
                    continue

                for serie in resultado['series']:
                    if 'localidade' not in serie or 'serie' not in serie:
                        continue

                    localidade = serie['localidade']['nome']
                    for periodo, valor in serie['serie'].items():
                        ano = periodo[:4]
                        mes = periodo[4:]
                        try:
                            valor_float = float(valor)
                        except ValueError:
                            valor_float = None

                        rows.append({
                            "Categoria": current_categoria if current_categoria else 'Categoria Desconhecida',
                            "Ano": ano,
                            "Mês": mes,
                            "Localidade": localidade,
                            "Valor Índice": valor_float
                        })

    df = pd.DataFrame(rows)
    df['Categoria'] = df['Categoria'].ffill()
    return df

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = process_pesquisa_industrial_json(data)

    if not df.empty:
        print(df.head())
        print("Salvando o arquivo CSV...")
        df.to_csv("pesquisa_industrial.csv", index=False, encoding='utf-8')
        print("Arquivo CSV salvo com sucesso!")
    else:
        print("O DataFrame está vazio, nenhum arquivo será salvo.")
else:
    print("Erro na requisição:", response.status_code, response.text)


                Categoria   Ano Mês Localidade  Valor Índice
0  Categoria Desconhecida  2019  01     Brasil      98.33759
1  Categoria Desconhecida  2019  02     Brasil      90.74466
2  Categoria Desconhecida  2019  03     Brasil      91.47444
3  Categoria Desconhecida  2019  04     Brasil      93.34245
4  Categoria Desconhecida  2019  05     Brasil     101.14052
Salvando o arquivo CSV...
Arquivo CSV salvo com sucesso!
